In [1]:
!pip install PyPDF2
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [7]:
import requests, io, PyPDF2
import pandas as pd
import spacy
import string, re
import gensim

from sklearn.feature_extraction import text
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn import preprocessing
import wordcloud

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:

urlMap = {
    "barclays": "https://home.barclays/content/dam/home-barclays/documents/citizenship/ESG/Barclays-PLC-ESG-Report-2019.pdf",
    "jp morgan chase": "https://impact.jpmorganchase.com/content/dam/jpmc/jpmorgan-chase-and-co/documents/jpmc-cr-esg-report-2019.pdf",
    "morgan stanley": "https://www.morganstanley.com/pub/content/dam/msdotcom/sustainability/Morgan-Stanley_2019-Sustainability-Report_Final.pdf",
    "goldman sachs": "https://www.goldmansachs.com/what-we-do/sustainable-finance/documents/reports/2019-sustainability-report.pdf",
    "hsbc": "https://www.hsbc.com/-/files/hsbc/our-approach/measuring-our-impact/pdfs/190408-esg-update-april-2019-eng.pdf",
    "citi": "https://www.citigroup.com/citi/about/esg/download/2019/Global-ESG-Report-2019.pdf",
    "td bank": "https://www.td.com/document/PDF/corporateresponsibility/2018-ESG-Report.pdf",
    "bank of america": "https://about.bankofamerica.com/assets/pdf/Bank-of-America-2017-ESG-Performance-Data-Summary.pdf",
    "rbc": "https://www.rbc.com/community-social-impact/_assets-custom/pdf/2019-ESG-Report.PDF",
    "macquarie": "https://www.macquarie.com/assets/macq/investor/reports/2020/sections/Macquarie-Group-FY20-ESG.pdf",
    "lloyds": "https://www.lloydsbankinggroup.com/globalassets/documents/investors/2020/2020feb20_lbg_esg_approach.pdf",
    "santander": "https://www.santander.co.uk/assets/s3fs-public/documents/2019_santander_esg_supplement.pdf",
    "bluebay": "https://www.bluebay.com/globalassets/documents/bluebay-annual-esg-investment-report-2018.pdf",
    "lasalle": "https://www.lasalle.com/documents/ESG_Policy_2019.pdf",
    "riverstone": "https://www.riverstonellc.com/media/1196/riverstone_esg_report.pdf",
    "aberdeen standard": "https://www.standardlifeinvestments.com/RI_Report.pdf",
    "apollo": "https://www.apollo.com/~/media/Files/A/Apollo-V2/documents/apollo-2018-esg-summary-annual-report.pdf",
    "bmogan": "https://www.bmogam.com/gb-en/intermediary/wp-content/uploads/2019/02/cm16148-esg-profile-and-impact-report-2018_v33_digital.pdf",
    "vanguard": "https://personal.vanguard.com/pdf/ISGESG.pdf",
    "ruffer": "https://www.ruffer.co.uk/-/media/Ruffer-Website/Files/Downloads/ESG/2018_Ruffer_report_on_ESG.pdf",
    "northern trust": "https://cdn.northerntrust.com/pws/nt/documents/fact-sheets/mutual-funds/institutional/annual-stewardship-report.pdf",
    "hermes investments": "https://www.hermes-investment.com/ukw/wp-content/uploads/sites/80/2017/09/Hermes-Global-Equities-ESG-Dashboard-Overview_NB.pdf",
    "abris capital": "http://www.abris-capital.com/sites/default/files/Abris%20ESG%20Report%202018.pdf",
    "schroders": "https://www.schroders.com/en/sysglobalassets/digital/insights/2019/pdfs/sustainability/sustainable-investment-report/sustainable-investment-report-q2-2019.pdf",
    "lazard": "https://www.lazardassetmanagement.com/docs/-m0-/54142/LazardESGIntegrationReport_en.pdf",
    "credit suisse": "https://www.credit-suisse.com/pwp/am/downloads/marketing/br_esg_capabilities_uk_csam_en.pdf",
    "coller capital": "https://www.collercapital.com/sites/default/files/Coller%20Capital%20ESG%20Report%202019-Digital%20copy.pdf",
    "cinven": "https://www.cinven.com/media/2086/81-cinven-esg-policy.pdf",
    "warburg pincus": "https://www.warburgpincus.com/content/uploads/2019/07/Warburg-Pincus-ESG-Brochure.pdf",
    "exponent": "https://www.exponentpe.com/sites/default/files/2020-01/Exponent%20ESG%20Report%202018.pdf",
    "silverfleet capital": "https://www.silverfleetcapital.com/media-centre/silverfleet-esg-report-2020.pdf",
    "kkr": "https://www.kkr.com/_files/pdf/KKR_2018_ESG_Impact_and_Citizenship_Report.pdf",
    "cerberus": "https://www.cerberus.com/media/2019/07/Cerberus-2018-ESG-Report_FINAL_WEB.pdf",
    "standard chartered": "https://av.sc.com/corp-en/others/2018-sustainability-summary2.pdf"
}

In [4]:
def extractContent(url):
    try:
        response = requests.get(url)
        pdf = PyPDF2.PdfReader(io.BytesIO(response.content))  # Use PdfReader instead of PdfFileReader
        text = [pdf.pages[ii].extract_text() for ii in range(len(pdf.pages))]  # Adjust method to extract text
        return "\n".join(text)
    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")
        return ""


In [5]:

contentMap = {x: extractContent(urlMap[x]) for x in urlMap}
contentMap = {x: contentMap[x] for x in contentMap if len(contentMap[x]) > 0}
{x: len(contentMap[x]) for x in contentMap}

Error processing URL https://www.td.com/document/PDF/corporateresponsibility/2018-ESG-Report.pdf: PyCryptodome is required for AES algorithm
Error processing URL https://about.bankofamerica.com/assets/pdf/Bank-of-America-2017-ESG-Performance-Data-Summary.pdf: EOF marker not found
Error processing URL https://www.lloydsbankinggroup.com/globalassets/documents/investors/2020/2020feb20_lbg_esg_approach.pdf: EOF marker not found
Error processing URL https://www.bluebay.com/globalassets/documents/bluebay-annual-esg-investment-report-2018.pdf: EOF marker not found
Error processing URL https://www.lasalle.com/documents/ESG_Policy_2019.pdf: EOF marker not found
Error processing URL https://www.standardlifeinvestments.com/RI_Report.pdf: EOF marker not found
Error processing URL https://www.apollo.com/~/media/Files/A/Apollo-V2/documents/apollo-2018-esg-summary-annual-report.pdf: EOF marker not found
Error processing URL https://www.bmogam.com/gb-en/intermediary/wp-content/uploads/2019/02/cm16148-

{'barclays': 403226,
 'jp morgan chase': 123318,
 'morgan stanley': 132438,
 'goldman sachs': 176124,
 'hsbc': 199845,
 'citi': 453885,
 'rbc': 146872,
 'macquarie': 78517,
 'santander': 67115,
 'riverstone': 42127,
 'ruffer': 124995,
 'abris capital': 2686,
 'schroders': 64061,
 'lazard': 23518,
 'cinven': 17749,
 'exponent': 32366,
 'kkr': 108422,
 'standard chartered': 68409}

In [6]:
# List of URLs with "EOF marker not found" error
urls_with_eof_errors = [
    "https://www.td.com/document/PDF/corporateresponsibility/2018-ESG-Report.pdf",
    "https://about.bankofamerica.com/assets/pdf/Bank-of-America-2017-ESG-Performance-Data-Summary.pdf",
    "https://www.lloydsbankinggroup.com/globalassets/documents/investors/2020/2020feb20_lbg_esg_approach.pdf",
    "https://www.bluebay.com/globalassets/documents/bluebay-annual-esg-investment-report-2018.pdf",
    "https://www.lasalle.com/documents/ESG_Policy_2019.pdf",
    "https://www.standardlifeinvestments.com/RI_Report.pdf",
    "https://www.apollo.com/~/media/Files/A/Apollo-V2/documents/apollo-2018-esg-summary-annual-report.pdf",
    "https://www.bmogam.com/gb-en/intermediary/wp-content/uploads/2019/02/cm16148-esg-profile-and-impact-report-2018_v33_digital.pdf",
    "https://personal.vanguard.com/pdf/ISGESG.pdf",
    "https://www.hermes-investment.com/ukw/wp-content/uploads/sites/80/2017/09/Hermes-Global-Equities-ESG-Dashboard-Overview_NB.pdf",
    "https://www.credit-suisse.com/pwp/am/downloads/marketing/br_esg_capabilities_uk_csam_en.pdf",
    "https://www.collercapital.com/sites/default/files/Coller%20Capital%20ESG%20Report%202019-Digital%20copy.pdf",
    "https://www.warburgpincus.com/content/uploads/2019/07/Warburg-Pincus-ESG-Brochure.pdf",
    "https://www.silverfleetcapital.com/media-centre/silverfleet-esg-report-2020.pdf",
    "https://www.cerberus.com/media/2019/07/Cerberus-2018-ESG-Report_FINAL_WEB.pdf"
]

# Remove the problematic URLs from the urlMap
for url in urls_with_eof_errors:
    urlMap = {k: v for k, v in urlMap.items() if v != url}

# Return the updated number of URLs in urlMap
len(urlMap)


19

In [8]:

spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
def extractStatements(nlp, text):
    printable = set(string.printable)
    text = "".join(filter(lambda x: x in printable, text))

    lines = []
    prev = ""
    for line in text.split("\n"):
        if(line.startswith(" ") or not prev.endswith(".")):
            prev = prev + " " + line
        else:
            lines.append(prev)
            prev = line
    lines.append(prev)

    sentences = []
    for line in lines:
        line = re.sub(r'^\s?\d+(.*)$', r'\1', line)
        line = line.strip()
        line = re.sub('\s?-\s?', '-', line)
        line = re.sub(r'\s?([,:;\.])', r'\1', line)
        line = re.sub(r'\d{5,}', r' ', line)
        line = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', r' ', line)
        line = re.sub('\s+', ' ', line)

        for part in list(nlp(line).sents):
            sentences.append(str(part).strip())

    return sentences

In [10]:

def lemmatize(nlp, text):
    doc = nlp(text)
    lemma = []
    for token in doc:
        if(token.lemma_ not in ["-PRON-"]):
            lemma.append(token.lemma_)

    gen = gensim.utils.simple_preprocess(" ".join(lemma), deacc=True)
    return " ".join(gen)

In [11]:

esgDf = pd.DataFrame()
for ii in contentMap:
    stmtVar = extractStatements(nlp, contentMap[ii])
    stmtVar = list(filter(lambda x: len(x) > 100, stmtVar))
    lemmaVar = list(map(lambda x: lemmatize(nlp, x), stmtVar))
    newDf = pd.DataFrame({"name": ii, "stmt": stmtVar, "lemma": lemmaVar})
    esgDf = esgDf.append(newDf, ignore_index=True)

<ipython-input-11-35c554b20c10>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  esgDf = esgDf.append(newDf, ignore_index=True)
<ipython-input-11-35c554b20c10>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  esgDf = esgDf.append(newDf, ignore_index=True)
<ipython-input-11-35c554b20c10>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  esgDf = esgDf.append(newDf, ignore_index=True)
<ipython-input-11-35c554b20c10>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  esgDf = esgDf.append(newDf, ignore_index=True)
<ipython-input-11-35c554b20c10>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

In [12]:
esgDf

,name,stmt,lemma
0,barclays,Barclays PLC Environmental Social Governance R...,barclays plc environmental social governance r...
1,barclays,Barclays is a British universal bank Our Purpo...,barclays be british universal bank our purpose...
2,barclays,"For over 325 years we have funded progress, an...",for over year we have fund progress and today ...
3,barclays,FOR OUR CUSTOMERS AND CLIENTS We will help the...,for our customer and client we will help they ...
4,barclays,FOR SOCIETY Our success over the long term is ...,for society our success over the long term be ...
...,...,...,...
9091,standard chartered,Waste/FTE (kg/headcount/year) Waste reused or ...,waste fte kg headcount year waste reuse or rec...
9092,standard chartered,This refects the Groups headcount (formerly la...,this refect the groups headcount formerly labe...
9093,standard chartered,"As a result, we have restated Scope 3 emission...",as result we have restate scope emission for a...
9094,standard chartered,Scaled-up waste data is not representative and...,scale up waste datum be not representative and...


In [13]:

fsiStopWords = ["plc", "group", "target", "track", "capital", "holding",
    "report", "annualreport", "esg", "bank", "report", "long", "make"]

for nm in esgDf.name.unique():
    for ii in nm.split(" "):
        fsiStopWords.append(ii)

stopWords = text.ENGLISH_STOP_WORDS.union(fsiStopWords)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Compute the term frequencies for bigrams across entire corpus
tf_vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=list(stopWords))
tf_matrix = tf_vectorizer.fit_transform(esgDf.lemma)
tf_sum = np.sum(tf_matrix, axis=0)
tf_bigrams = [(word, tf_sum[0, idx]) for word, idx in tf_vectorizer.vocabulary_.items()]
tf_bigrams = sorted(tf_bigrams, key=lambda x: x[1], reverse=True)
top_tf_bigrams = tf_bigrams[:3]

large_banks =[
    "barclays",
    "jp morgan chase",
    "morgan stanley",
    "goldman sachs",
    "hsbc",
    "citi",
    "rbc",
    "macquarie",
    "santander",
    "standard chartered"
]

filtered_df = esgDf[esgDf.name.isin(large_banks)]

# Compute TF-IDF for bigrams in the filtered dataset
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 2), stop_words=list(stopWords))
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df.lemma)
tfidf_sum = np.sum(tfidf_matrix, axis=0)
tfidf_bigrams = [(word, tfidf_sum[0, idx]) for word, idx in tfidf_vectorizer.vocabulary_.items()]
tfidf_bigrams = sorted(tfidf_bigrams, key=lambda x: x[1], reverse=True)
top_tfidf_bigrams = tfidf_bigrams[:3]

In [15]:
print("What are the three most common bigrams by TF alone?")
top_tf_bigrams

What are the three most common bigrams by TF alone?


[('environmental social', 521),
 ('climate change', 445),
 ('social governance', 275)]

In [16]:
print("What are the three most common bigrams (by TF-IDF) among the reports by (only) large banks?")
top_tfidf_bigrams

What are the three most common bigrams (by TF-IDF) among the reports by (only) large banks?


[('environmental social', 50.57102215303408),
 ('climate change', 39.89971734801227),
 ('risk management', 31.888391778512283)]